In [1]:
import pickle
import numpy as np

## VGG19: Model Preparation

In [2]:
from keras.applications.vgg19 import VGG19
model = VGG19(include_top=False)
model.summary()

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

80142336/80134624 [==============================] - 2s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None

In [4]:
#cnn_output=model.predict(data)

## Fully-Connected Layer Tuning 

In [14]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential


def train_model_neural_net(X_train,Y_train,X_val,Y_val):
    mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=8)
    print(X_train.shape[1:])
    model = Sequential()
    #model.add(GlobalAveragePooling2D(input_shape=(2, 2, 512)))
    model.add(GlobalAveragePooling2D(input_shape=X_train.shape[1:]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model.fit(X_train,Y_train ,validation_data=(X_val,Y_val),epochs=50,shuffle=True,batch_size=256,callbacks=[es, mc], verbose=0)
    saved_model = load_model('best_model.h5')
    train_pred =saved_model.predict(X_train)            
    valid_pred =saved_model.predict(X_val)
    train_loss, train_acc = saved_model.evaluate(X_train,Y_train, verbose=0)
    val_loss, val_acc = saved_model.evaluate(X_val,Y_val, verbose=0)
    print('accuracy Train: %.3f, Test: %.3f' % (train_acc, val_acc))
    print('loss Train: %.3f, Test: %.3f' % (train_loss, val_loss))
    #test_pred = saved_model.predict(X_test)
    return train_pred ,valid_pred,saved_model,val_acc,val_loss

def train_kfold_neural_net(X_train,Y_train,split=5):
    final_train_pred=np.zeros((X_train.shape[0],4))
    from sklearn.model_selection import StratifiedKFold
    kf =  StratifiedKFold(n_splits=split, shuffle=False)
    i=1
    score=0
    accuracies=np.array([])
    losses=np.array([])
    for train_index, val_index in kf.split(X_train, Y_train):
        print("fold:"+str(i))
        train_fold_features, val_fold_features = X_train[train_index], X_train[val_index]
        train_fold_target, val_fold_target = Y_train[train_index], Y_train[val_index] 
        
        train_pred ,valid_pred,model,val_acc,val_loss=train_model_neural_net( 
                                                        X_train=train_fold_features,
                                                        Y_train= train_fold_target,
                                                        X_val= val_fold_features,
                                                        Y_val= val_fold_target
                                                    )
        accuracies=np.append(accuracies,val_acc)
        losses=np.append(losses,val_loss)
        final_train_pred[val_index]=valid_pred
        i=i+1
    avg_acc=accuracies.sum() / 5
    avg_loss=losses.sum() / 5
    print('overall performance avg_acc: %.3f, avg_loss: %.3f' % (avg_acc, avg_loss))
    return final_train_pred,model,avg_acc,avg_loss

final_train_pred,model,avg_acc,avg_loss=train_kfold_neural_net(cnn_output,Y_train=labels,split=5)

fold:1
(7, 7, 512)
accuracy Train: 0.925, Test: 0.843
loss Train: 0.266, Test: 0.516
fold:2
(7, 7, 512)
accuracy Train: 0.913, Test: 0.821
loss Train: 0.297, Test: 0.566
fold:3
(7, 7, 512)
accuracy Train: 0.921, Test: 0.836
loss Train: 0.257, Test: 0.570
fold:4
(7, 7, 512)
accuracy Train: 0.900, Test: 0.825
loss Train: 0.318, Test: 0.521
fold:5
(7, 7, 512)
accuracy Train: 0.923, Test: 0.804
loss Train: 0.297, Test: 0.669
overall performance avg_acc: 0.826, avg_loss: 0.568
